In [1]:
import pandas as pd
filename = r'C:\Users\Danny\nbagames1985to2019.csv'
df = pd.read_csv(filename, index_col=None, header=0)
df.head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,21985,1610612737,ATL,Atlanta Hawks,28500005,1985-10-25,ATL vs. WAS,L,240,41,...,28,44,25,8,5,16,23,91,-9,0
1,21985,1610612764,WAS,Washington Bullets,28500005,1985-10-25,WAS @ ATL,W,240,40,...,29,40,21,11,7,17,19,100,9,0


In [2]:
df.loc[df['FGA'] == 0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
63090,22012,1610612754,IND,Indiana Pacers,21201214,2013-04-16,IND @ BOS,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
63091,22012,1610612738,BOS,Boston Celtics,21201214,2013-04-16,BOS vs. IND,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.drop([63090, 63091], inplace=True)

In [4]:
df.drop(['SEASON_ID', 'TEAM_ID', 'TEAM_NAME', 'GAME_ID',
         'MATCHUP', 'FTM', 'FT_PCT', 'AST', 'VIDEO_AVAILABLE'], inplace=True, axis=1)

In [5]:
# This code gives a count and list of all teams (&abbreviations) during the time spans
teamnames = df['TEAM_ABBREVIATION']
names=[]
count=0
for name in teamnames:
    if name in names:
        continue
    else:
        count+=1
        names.append(name)
print(count)
print(names)

40
['ATL', 'WAS', 'DEN', 'GOS', 'DET', 'MIL', 'LAC', 'SAC', 'HOU', 'UTH', 'BOS', 'NJN', 'PHX', 'POR', 'CLE', 'CHI', 'IND', 'SEA', 'DAL', 'SAN', 'LAL', 'PHL', 'NYK', 'CHH', 'MIA', 'MIN', 'ORL', 'VAN', 'TOR', 'UTA', 'PHI', 'SAS', 'GSW', 'MEM', 'NOH', 'CHA', 'NOK', 'OKC', 'BKN', 'NOP']


In [6]:
# This code changes WinLoss column from WL to binary
df.loc[df.WL == 'W', "WL"] = 1
df.loc[df.WL == 'L', "WL"] = 0

In [7]:
grouped = df.groupby('MIN')
for key, mins in grouped:
    print(key)
    mins.head()

216
240
265
290
315
340
365


In [8]:
df.loc[df['MIN'] == 216]

,TEAM_ABBREVIATION,GAME_DATE,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTA,OREB,DREB,REB,STL,BLK,TOV,PF,PTS,PLUS_MINUS
23513,DEN,1996-11-10,0,216,30,63,0.476,6,12,0.5,14,11,28,39,5,4,23,23,79,-29


In [9]:
df.loc[23513, 'MIN']=240

In [10]:
col_names =['FGM', 'FGA', 'FG3M', 'FG3A','FTA', 'OREB', 'DREB', 'REB', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']
#normalizing even indices
for index in range(31545):
    for mins in [265,290,315,340,365]:
        weighting = 240/mins
        if df.loc[2*index, 'MIN'] == mins:
            for col_name in col_names:
                df.loc[2*index, col_name] = round((df.loc[2*index, col_name])*weighting)       
for index in [31546,38942]:
    for mins in [265,290,315,340,365]:
        weighting = 240/mins
        if df.loc[2*index, 'MIN'] == mins:
            for col_name in col_names:
                df.loc[2*index, col_name] = round((df.loc[2*index, col_name])*weighting)
#normalizing odd indices
for index in range(31545):
    for mins in [265,290,315,340,365]:
        weighting = 240/mins
        if df.loc[2*index+1, 'MIN'] == mins:
            for col_name in col_names:
                df.loc[2*index+1, col_name] = round((df.loc[2*index+1, col_name])*weighting)       
for index in [31546,38942]:
    for mins in [265,290,315,340,365]:
        weighting = 240/mins
        if df.loc[2*index+1, 'MIN'] == mins:
            for col_name in col_names:
                df.loc[2*index+1, col_name] = round((df.loc[2*index+1, col_name])*weighting)                


In [11]:
df['OPP_PTS']=df['PTS']-df['PLUS_MINUS']

In [12]:
df['FG_PCT']=round(df['FGM']/df['FGA'],2)
df['FG3_PCT']=round(df['FG3M']/df['FG3A'],2)
df['FG3_PCT']=df['FG3_PCT'].fillna(0)

In [13]:
for index in range(31545):
    df.loc[2*index, 'REB_RATE'] = round(df.loc[2*index, 'DREB']/(df.loc[2*index, 'DREB']+df.loc[2*index+1, 'OREB']),2)
    df.loc[2*index+1, 'REB_RATE'] = round(df.loc[2*index+1, 'DREB']/(df.loc[2*index+1, 'DREB']+df.loc[2*index, 'OREB']),2)
for index in [31546,38942]:
    df.loc[2*index, 'REB_RATE'] = round(df.loc[2*index, 'DREB']/(df.loc[2*index, 'DREB']+df.loc[2*index+1, 'OREB']),2)
    df.loc[2*index+1, 'REB_RATE'] = round(df.loc[2*index+1, 'DREB']/(df.loc[2*index+1, 'DREB']+df.loc[2*index, 'OREB']),2)

In [14]:
for index in range(31545):
    df.loc[2*index, 'OPP_FG_PCT'] = df.loc[2*index+1, 'FG_PCT']
    df.loc[2*index+1, 'OPP_FG_PCT'] = df.loc[2*index, 'FG_PCT']
for index in [31546,38942]:
    df.loc[2*index, 'OPP_FG_PCT'] = df.loc[2*index+1, 'FG_PCT']
    df.loc[2*index+1, 'OPP_FG_PCT'] = df.loc[2*index, 'FG_PCT']

In [15]:
for index in range(31545):
    df.loc[2*index, 'OPP_3FG_PCT'] = df.loc[2*index+1, 'FG3_PCT']
    df.loc[2*index+1, 'OPP_3FG_PCT'] = df.loc[2*index, 'FG3_PCT']
for index in [31546,38942]:
    df.loc[2*index, 'OPP_3FG_PCT'] = df.loc[2*index+1, 'FG3_PCT']
    df.loc[2*index+1, 'OPP_3FG_PCT'] = df.loc[2*index, 'FG3_PCT']
df.head(2)

,TEAM_ABBREVIATION,GAME_DATE,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,OPP_PTS,REB_RATE,OPP_FG_PCT,OPP_3FG_PCT
0,ATL,1985-10-25,0,240,41,92,0.45,0,3,0.0,...,8,5,16,23,91,-9,100,0.72,0.51,0.0
1,WAS,1985-10-25,1,240,40,78,0.51,0,2,0.0,...,11,7,17,19,100,9,91,0.64,0.45,0.0


In [16]:
df.to_csv( "nbagames1985to2019_cleaned.csv", index=False, encoding='utf-8-sig')